In [1]:
# !pip install konlpy

In [2]:
import re
from konlpy.tag import Okt
from collections import Counter
print("임포트 완료")

In [3]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

## 영어 Word2Vec 실습과 OOV 문제

영어 Word2Vec 실습
----

영어 데이터를 다운로드받아 직접 Word2Vec을 훈련시켜보겠습니다. Word2Vec을 별도로 구현할 필요없이 파이썬의 gensim 패키지를 통해 이미 구현된 Word2Vec 모델을 사용할 수 있습니다. 여기서 사용할 훈련 데이터는 NLTK에서 제공하는 코퍼스이며, gensim 패키지는 토픽 모델링을 위한 NLP 패키지입니다.

Cloud 사용자는 이미 설치되어 있으니 아래의 명령어를 참고로만 알아두세요.

    pip install nltk
    pip install gensim

In [5]:
import nltk
nltk.download('abc')
nltk.download('punkt')

In [6]:
from nltk.corpus import abc
corpus = abc.sents()
print("슝~")

In [7]:
print(corpus[:3])

In [8]:
print('코퍼스의 크기 :',len(corpus))

In [9]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

위 코드에서 각 파라미터가 의미하는 바는 아래와 같습니다.

- vector size = 학습 후 임베딩 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않아요.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBoW, 1은 Skip-gram.

아주 잠깐의 기다림 끝에 Word2Vec의 학습이 완료됩니다. Word2Vec는 입력한 단어에 대해서 가장 코사인 유사도가 높은 단어들을 출력하는 model.wv.most_similar를 지원합니다. 'man'과 가장 유사한 단어들은 어떤 단어들일까요?

In [10]:
model_result = model.wv.most_similar("man")
print(model_result)

In [12]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('./w2v') 
loaded_model = KeyedVectors.load_word2vec_format("./w2v")
print("모델  load 완료!")

## FastText
- subword 단위로 쪼개서 오타에 강건함

In [13]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

In [14]:
fasttext_model.wv.most_similar('overacting')

In [15]:
fasttext_model.wv.most_similar('memoryy')

## Glove 실습

In [16]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

In [17]:
glove_model.most_similar('overacting')

In [18]:
glove_model.most_similar('memoryy')

GloVe는 Word2Vec과 같이 OOV 문제를 가지고 있어서 'memoryy'라는 단어는 인식하지 못합니다. 또한 pre-trained GloVe 모델은 한글이나 알파벳 대문자가 포함된 데이터셋으로 학습하지 않았기 때문에 알파벳 소문자만 인식한다는 사실에 유의하세요!